In [1]:
# !pip install googletrans
# !pip install pydub
# !pip3 install SpeechRecognition
# !pip install librosa

'utf8' is not recognized as an internal or external command,
operable program or batch file.


'utf8' is not recognized as an internal or external command,
operable program or batch file.


'utf8' is not recognized as an internal or external command,
operable program or batch file.
You should consider upgrading via the 'c:\users\juvin\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.


'utf8' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
# !pip install flask

In [2]:
"""Develop an api"""

import librosa
import spacy
import re
from pydub import AudioSegment
from pydub.silence import split_on_silence
import warnings
import en_core_web_sm
from spacy.util import load_model
warnings.filterwarnings('ignore')
import pandas as pd
import speech_recognition as sr
import os
from flask import Flask, request, jsonify, render_template, flash, redirect, url_for, send_from_directory
import numpy as np
import json
from googletrans import Translator
from werkzeug.wrappers import Request, Response
from werkzeug.utils import secure_filename
from tensorflow.keras.models import load_model
import tensorflow as tf

present_tenses = ['VBP', 'VBZ','VBG']
nlp = en_core_web_sm.load()

def extract_spectrum(file_name, file_length=5, bin_factor=4, **kwargs):
    """
    file_length is the target time length of the file in seconds
    bin_factor is the amount by which we enlarge the time bins from 512 samples:
    Extract mel spectrogram from audio file `file_name`
    """

    X, sample_rate = librosa.core.load(file_name)
    file_name = file_name.split('/')[-1]
    target_name = file_name[:-4]
    target_length = int(file_length * sample_rate)
    target = np.zeros(target_length)
    if X.shape[0] >= target_length:
        target = X[:target_length]
    else:
        target[:X.shape[0]] = X
    X = target
    mel = librosa.feature.melspectrogram(X, sr=sample_rate, hop_length=512 * bin_factor).T
    print(file_name, ' loaded with shape ', mel.shape)
    return mel


def open_model(filename="best_model_final.h5"):
    """ Given filename, returns the model
    param filename: filepath for model
    return : pretrained model """
    with tf.device("/cpu:0"):
        model = load_model(filename)
    return model

def predict(X):
    """ Given X, returns a prediction
    param X: the feature values to consider
    returns pred_pickle: the prediction """

    X = extract_spectrum(X).reshape(-1,54,128,1)
    with tf.device("/cpu:0"):
        pred = model.predict(X)
        if pred[0][0] <= 0.5:
            return 'female'
        else:
            return 'male'

def get_large_audio_transcription(X):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    r = sr.Recognizer()
    # open the audio file using pydub
    sound = AudioSegment.from_wav(X)
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                whole_text += text
    # return the text for all chunks detected
    return whole_text



def determine_chunks(pronouns, vals, text,present_tenses = present_tenses):
    nlp = spacy.load('en')

    present_tenses = ['VBP', 'VBZ','VBG']

    chunks = []

    iterations = len(pronouns)

    for i in range(iterations):

        start = vals[i]
        stop = None

        if i != (iterations - 1):
            stop = vals[i+1]

        if pronouns[i] != 'I':
            chunks.append((str(text[start:stop]),0))
            continue


        present_verbs_change = [word for word in text[start:stop]
                                if word.tag_ in
                                present_tenses]

        if len(present_verbs_change) == 0:
            chunks.append((str(text[start:stop]),0))
            continue

        chunks.append((str(text[start:stop]),1))

    return chunks

def determine_tense_input(sentence, present_tenses = present_tenses):

    """ Seperates words into chunks based on subjects of the sentence
    and sends them to the determine_chunks function to determine
    if that chunk requires further manipulation or not
    param ....
    .....
    ....
    returns: list of tuples (portion, int) where 1 means the chunk
    requires further manipulation and 0 means it can be left as is """

    text = nlp(sentence)

    present = len([word for word in text
                    if word.tag_ in present_tenses])

    if present == 0 :
        return [(sentence,0)]

    sub_toks = list(sorted([ (i, tok) for i, tok
                            in enumerate(text)
                            if tok.dep_ == "nsubj" ]))


    vals = [x[0] for x in sub_toks]
    pronouns = [str(x[1]) for x in sub_toks]

    if 'I' not in pronouns:
        return [(sentence,0)]

    return determine_chunks(pronouns, vals, text)

def get_translation(output, gender):
    heb_I =  'אני'
    heb_she = 'היא'
    heb_he = 'הוא'
    ind = []
    translate = ''
    lenght = 0
    translator = Translator()
    replacer = 'I'


    for sentence, val in output:
        res = translator.translate(sentence, dest='he').text
        res_list = re.findall(r"[\w]+|['.,!?;]", res)
        if val>0:
            if gender == 'female':
                replacer = 'she'
                heb_replacer = heb_she
            elif gender == 'male':
                replacer = 'he'
                heb_replacer = heb_he
            index = [i for i,x in enumerate(res_list) if x==heb_I][0]
            ind.append(index+lenght)
        lenght += len(res_list)
        sentence_list = re.findall(r"[\w]+|['.,!?;]", sentence)
        sentence = ' '.join([replacer if word == 'I' else word for word in sentence_list])
        translate += " " + sentence
    gender_res = translator.translate(translate, dest='he').text
    gender_res_list = re.findall(r"[\w]+|['.,!?;]", gender_res)
    for i in ind:
        if gender_res_list[i] == heb_replacer:
            gender_res_list[i] = heb_I
        elif gender_res_list[i-1] ==  heb_replacer:
            gender_res_list[i-1] = heb_I
        elif gender_res_list[i+1] ==  heb_replacer:
            gender_res_list[i+1] = heb_I

    return ' '.join(gender_res_list)



app = Flask(__name__)

app.secret_key = 'some secret'

'''limiting the receiving file to 2 MB maximum'''
app.config['MAX_CONTENT_LENGTH'] = 2 * 1024 * 1024

UPLOAD_FOLDER = '~/Desktop/ITC_Final_Project/ITC_FinalProject_Gender_Voice_Recognition/yuval/uploads'
'''configuring uploading folder'''
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

'''setting accepted files to the server'''
ALLOWED_EXTENSIONS = {'wav', 'mp3'}

os.chdir('C:/Users/Juvin/Desktop/ITC_Final_Project/ITC_FinalProject_Gender_Voice_Recognition/Serah (next step)')
model = open_model()

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'],
                               filename)

@app.route("/prediction", methods = ['POST'])
def prediction():
    if request.method == 'POST':
        
        # check if the post request has the file part
        if 'file' not in request.files:
            print('no!')
            flash('No file part')
            return redirect(request.url)
        
        file = request.files['file']
        # if user does not select file, browser also
        # submit an empty part without filename
        if file.filename == '':
            print('yes!')
            flash('No selected file')
            return redirect(request.url)
        
        if file and allowed_file(file.filename):
            print('awesome!')   
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
#             return redirect(url_for('uploaded_file',
#                                     filename=filename))
        '''predict gender'''
        pred = predict(filename)
        '''speech to text'''
        chunks = get_large_audio_transcription(filename)
        split_sentence = determine_tense_input(chunks)
        '''modify translation as necessary'''
        output = get_translation(split_sentence, pred)
        '''prep a json to delivery'''
        json = {'pred':pred, 'output':output}
    
        return json
  


In [ ]:
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)


 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Sep/2020 18:12:10] "POST /prediction HTTP/1.1" 302 -


no!


127.0.0.1 - - [25/Sep/2020 18:12:11] "GET /prediction HTTP/1.1" 405 -
